<a href="https://colab.research.google.com/github/elibtronic/green_oa_compiler/blob/main/ORCID_API_Explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Green OA Compiler

Will build dataframe of ORCID and DOI of journal articles published from a date onward. Will need ORCID API key and secret to work

In [85]:
!pip install orcid

import requests
import json
import orcid
import pprint
import pandas as pd


from google.colab import files
from datetime import datetime

print("\nReady to proceed!")


Ready to proceed!


In [86]:
# @title ORCID List Creation Parameters {"run":"auto","vertical-output":true,"display-mode":"form"}
Client_ID = "paste in id" # @param {"type":"string"}
Client_Secret = "paste in secret" # @param {"type":"string"}

# @markdown Affiliation search string

s_query = "current-institution-affiliation-name:(\"Brock University\")"#@param {type:"string"}


# @markdown Date to start harvest from?
Start_Date = "2025-01-01" # @param {"type":"date"}
s_date = datetime.strptime(Start_Date, '%Y-%m-%d')

print("Options set!")

Options set!


In [91]:
#Using the ORCID library.
api = orcid.PublicAPI(Client_ID, Client_Secret, sandbox=False)
search_token = api.get_search_token_from_orcid()

#Build a dictionary of ORCIDs currently affiliated with Brock

b_members = []
search_results = api.search_generator(s_query, access_token=search_token)

for page in search_results:
  b_members.append(page['orcid-identifier']['path'])


In [88]:
#some way to augement that list if needed? -> maybe, current affil should fix that?

In [92]:
harvested_j_works = []
#check for most recent pub date of items associated with that orcid
#extract some metadata for that, doi, title, journal,
#created a df with orcid of person and doi etc

for b_orcid in b_members:
  works_list = api.read_record_public(b_orcid, 'works',search_token)

  for w in works_list['group']:
    date_work = datetime.fromtimestamp(w['last-modified-date']['value'] / 1000)
    if date_work > s_date:
      for ws in w.get("work-summary"):
        if ws.get('type') == 'JOURNAL_ARTICLE':
          eids = ws.get('external-ids')
          for e in eids['external-id']:
            if e.get('external-id-type') == 'doi':
              harvested_j_works.append([b_orcid,e.get('external-id-value')])


In [102]:
df = pd.DataFrame(harvested_j_works)
df.columns = ['ORCID',"DOI"]
df.drop_duplicates(subset=['DOI'],inplace=True)
df.reset_index(drop=True,inplace=True)

df

,ORCID,DOI
0,0000-0001-6297-4298,10.1108/JHOM-02-2023-0036
1,0000-0002-2899-0048,10.1139/cjps-2024-0160
2,0000-0002-3772-5561,10.1123/jpah.2024-0225
3,0000-0001-9889-5313,10.1126/sciadv.adr2613
4,0000-0001-8418-449X,10.14814/phy2.70299
...,...,...
779,0000-0002-0048-491X,10.1080/030144600419305
780,0000-0002-0048-491X,10.1002/(SICI)1520-6300(200005/06)12:3<395::AI...
781,0000-0002-0048-491X,10.1123/PES.11.3.208
782,0000-0002-0048-491X,10.1093/AJCN/69.6.1123


In [103]:
#Run to save as CSV
csv_file_name = "Brock_ORCID_Harvest_Start_Date_"+str(Start_Date)+".csv"
print("Saving to file: "+csv_file_name)
df.to_csv(csv_file_name)
files.download(csv_file_name)

Saving to file: Brock_ORCID_Harvest_Start_Date_2025-01-01.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>